In [1]:
import os
import os.path as osp

import torch
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

from ogb.nodeproppred import PygNodePropPredDataset


# create directory
folder_path = osp.join(os.getcwd(), 'data')

if not osp.exists(folder_path):
    os.makedirs(folder_path)
    print("Directory '% s' created" % folder_path)


/home/jharris/Desktop/approx_attention/venvs/CPU_venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
path = osp.join(folder_path, 'cora')

if not osp.exists(path):
    os.makedirs(path)

for K in range(6):

    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToUndirected(),
        T.AddSelfLoops(),
        T.SIGN(K)
    ])

    # download
    dataset = Planetoid(
        root=path,
        name='Cora',
        transform=transform,
        split='full',
    )

    filename = osp.join(path, f'cora_sign_k{K}.pth')
    torch.save(dataset, filename)




Processing...
Done!


In [4]:
path = osp.join(folder_path, 'pubmed')

if not osp.exists(path):
    os.makedirs(path)

for K in range(6):

    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToUndirected(),
        T.AddSelfLoops(),
        T.SIGN(K)
    ])

    # download
    dataset = Planetoid(
        root=path,
        name='Pubmed',
        transform=transform,
        split='full',
    )

    filename = osp.join(path, f'pubmed_sign_k{K}.pth')
    torch.save(dataset, filename)


Processing...
Done!


In [5]:
path = osp.join(folder_path, 'arxiv')

if not osp.exists(path):
    os.makedirs(path)

for K in range(6):

    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToUndirected(),
        T.AddSelfLoops(),
        T.SIGN(K)
    ])

    # download
    dataset = PygNodePropPredDataset(
        'ogbn-arxiv',
        root=path,
        transform=transform)

    filename = osp.join(path, f'arxiv_sign_k{K}.pth')
    torch.save(dataset, filename)
    del transform, dataset


Downloaded 0.08 GB: 100%|██████████| 81/81 [00:07<00:00, 11.33it/s]


Extracting /home/jharris/Desktop/approx_attention/setup/data/arxiv/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 14266.34it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 4798.97it/s]

Saving...



Done!


In [6]:
path = osp.join(folder_path, 'products')

if not osp.exists(path):
    os.makedirs(path)

for K in range(6):

    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToUndirected(),
        T.AddSelfLoops(),
        T.SIGN(K)
    ])

    # download
    dataset = PygNodePropPredDataset(
        'ogbn-products',
        root=path,
        transform=transform)

    filename = osp.join(path, f'products_sign_k{K}.pth')
    torch.save(dataset, filename)
    del transform, dataset


Downloaded 1.38 GB: 100%|██████████| 1414/1414 [07:57<00:00,  2.96it/s]


Extracting /home/jharris/Desktop/approx_attention/setup/data/products/products.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:02<00:00,  2.05s/it]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 4500.33it/s]


Saving...


Done!


In [9]:
def standardize_data(dataset, data_name: str):
    """ standardize format of data object """

    possible_datasets = ['cora', 'pubmed', 'products', 'arxiv']
    assert data_name.lower() in possible_datasets

    # extract relevant information
    data = dataset[0]
    data.dataset_name = data_name.lower()
    data.num_classes = dataset.num_classes
    data.num_nodes = data.num_nodes
    data.num_edges = data.num_edges
    data.num_node_features = data.num_node_features
    data.n_id = torch.arange(data.num_nodes)  # global node id

    # standardize mask -- node idx, not bool mask
    if hasattr(dataset, 'get_idx_split'):
        masks = dataset.get_idx_split()
        data.train_mask = masks['train']
        data.val_mask = masks['valid']
        data.test_mask = masks['test']

        data.y = data.y.flatten()
    else:
        data.train_mask = torch.where(data.train_mask)[0]
        data.val_mask = torch.where(data.val_mask)[0]
        data.test_mask = torch.where(data.test_mask)[0]

    return data


In [11]:
path = osp.join(folder_path, 'arxiv')

if not osp.exists(path):
    os.makedirs(path)

for name in ['arxiv','products']:

    dataset = PygNodePropPredDataset(
        f'ogbn-{name}',
        root=path)

    filename = osp.join(path, f'{name}.pth')
    torch.save(standardize_data(dataset, name), filename)
    del dataset

In [12]:
def transform_data(data, dataset, K):
    """ SIGN transform """

    if dataset == 'arxiv':
        t_list = [
            T.NormalizeFeatures(),
            T.ToUndirected(),
            T.AddSelfLoops(),
            T.SIGN(K)
        ]
    else:
        t_list = [
            T.NormalizeFeatures(),
            T.ToUndirected(),
            T.AddSelfLoops(),
            T.SIGN(K)
        ]

    return T.Compose(t_list)(data)

In [13]:
transform_data(torch.load(filename), 'arxiv', 2)